In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), "picopic/lib/python"))

import numpy as np

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader

In [ ]:
# service options
delimiter = '/'

## configuration options
data_path = '/home/cosmonaut/dev/pic/picopic/'

if os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

timestamp=2.85e-10
cmap='terrain_r'
image_interpolation = 'nearest'

autoselect = True
show_grid=False
aspect='equal'

guess_number_ticks = 20

cmp_r = 'E/r'
cmp_z = 'E/z'
cmp_beam = 'density'
specie_beam = 'beam_electrons'

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]

r_scale = (size[2] - size[0]) / reader.meta.geometry_grid[0]
z_scale = (size[3] - size[1]) / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = reader.meta.get_clim_estimation()
beam_density = reader.meta.beams[0].bunch_density
beam_density_delta = reader.meta.beams[0].bunch_density * 0.2

clim_r = [-clim_estimation, clim_estimation]
clim_z = [-clim_estimation, clim_estimation]
clim_beam = [0, beam_density + beam_density_delta]

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'
cbar_beam_axis_label = r'$m^{-3}$'

plot_name_r = r'$\mathbf{Electric\enspace Field\enspace E_r\enspace Component}$'
plot_name_z = r'$\mathbf{Electric\enspace Field\enspace E_z\enspace Component}$'
plot_name_beam = r'$\mathbf{Electron\enspace Beam\enspace Density\enspace \rho_{beam}}$'

In [ ]:
# define plot builder
plot = PlotBuilder(size[3] - size[1], size[2] - size[0],
                   fig_color=reader.meta.figure_color, 
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height, 
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   
                   x_ticklabel_end=reader.meta.geometry_size[1] * z_scale, 
                   y_ticklabel_end=reader.meta.geometry_size[0] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect=aspect, image_interpolation=image_interpolation,
                   guess_number_ticks=guess_number_ticks)

In [ ]:
# get data
data_r = []
data_z = []
data_beam = []

for probe in reader.meta.probes:
    frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if ( (probe.shape == 'rec') 
        and (probe.size[0] == size[0]) 
        and (probe.size[1] == size[1]) 
        and (probe.size[2] == size[2]) 
        and (probe.size[3] == size[3]) ):
        if probe.component == cmp_r:
            data_r = reader.rec("{}".format(cmp_r), size, frame)
        if probe.component == cmp_z:
            data_z = reader.rec("{}".format(cmp_z), size, frame)
        if probe.component == cmp_beam and probe.specie == specie_beam:
            data_beam = reader.rec("{}{}{}".format(cmp_beam, delimiter, specie_beam), size, frame)

# try bigger frames, if autoselect enabled
if (len(data_r) == 0 or len(data_z) == 0 or len(data_beam) == 0) and autoselect:
    for probe in reader.meta.probes:
        frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if ( (probe.shape == 'rec') 
            and (probe.size[0] <= size[0]) 
            and (probe.size[1] <= size[1]) 
            and (probe.size[2] >= size[2]) 
            and (probe.size[3] >= size[3]) ):
            if probe.component == cmp_r:
                data_r = reader.rec("{}".format(cmp_r), probe.size, frame)[size[0]:size[2], size[1]:size[3]]
            if probe.component == cmp_z:
                data_z = reader.rec("{}".format(cmp_z), probe.size, frame)[size[0]:size[2], size[1]:size[3]]
            if probe.component == cmp_beam and probe.specie == specie_beam:
                data_beam = reader.rec("{}{}{}".format(cmp_beam, delimiter, specie_beam),
                                       probe.size, frame)[size[0]:size[2], size[1]:size[3]]

In [ ]:
# add subplots
plot.add_subplot_cartesian_2d(plot_name_r, 311, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(plot_name_z, 312, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(plot_name_beam, 313, x_axe_label=x_axis_label, y_axe_label=y_axis_label)


# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add images
plot.add_image(plot_name_r, data_r, cmap=cmap, clim=clim_r)
plot.add_image(plot_name_z, data_z, cmap=cmap, clim=clim_z)
plot.add_image(plot_name_beam, data_beam, cmap=cmap, clim=clim_beam)


# add colorbars
plot.add_colorbar(plot_name_r, ticks=clim_r, title=cbar_axis_label)
plot.add_colorbar(plot_name_z, ticks=clim_z, title=cbar_axis_label)
plot.add_colorbar(plot_name_beam, ticks=clim_beam, title=cbar_beam_axis_label)


plot.show()